In [ ]:
#Stella's Notes: This is the .py file Stella is currently using to write the NLP code for the project.
#Need to do:
#Incorporate age 
#How to save cleaned/stemmed/tokenized lyrics as df? How to save the dicts/inverted indexes etc as well?
#How to prioritize certain words above others. Should I even do this? E.g if someone enter "happy Monday", should I prioritize happy or Monday?
#How to semantically relate words? E.g cold with frozen, etc
#Handle misspelled words

#Create a datasets for: cleaned lyrics, tokenized lyrics, lyric polarized score, other essential stuff.

import json
import os
from flask import Flask, render_template, request
from flask_cors import CORS
from helpers.MySQLDatabaseHandler import MySQLDatabaseHandler
import pandas as pd


spotify_df = pd.read_csv("backend/spotify-tracks-dataset.csv")
lyric_df = pd.read_csv("/Users/juanruzhang/4300/4300-JSON-wanderingmelody/backend/spotify_millsongdata.csv")


ModuleNotFoundError: No module named 'flask'

In [197]:
spotify_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [198]:
lyric_df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [199]:
lyric_df['text']

0        Look at her face, it's a wonderful face  \r\nA...
1        Take it easy with me, please  \r\nTouch me gen...
2        I'll never know why I had to go  \r\nWhy I had...
3        Making somebody happy is a question of give an...
4        Making somebody happy is a question of give an...
                               ...                        
57645    Irie days come on play  \r\nLet the angels fly...
57646    Power to the workers  \r\nMore power  \r\nPowe...
57647    all you need  \r\nis something i'll believe  \...
57648    northern star  \r\nam i frightened  \r\nwhere ...
57649    come in  \r\nmake yourself at home  \r\ni'm a ...
Name: text, Length: 57650, dtype: object

In [200]:
from typing import List, Tuple, Dict
from collections.abc import Callable
import numpy as np
import re

In [201]:
def tokenize(text: str) -> List[str]:
    """Returns a list of words that make up the text.
    
    Note: for simplicity, lowercase everything. Do not remove duplicate words.
    Requirement: Use Regex to satisfy this function
    
    Parameters
    ----------
    text : str
        The input string to be tokenized.

    Returns
    -------
    List[str]
        A list of strings representing the words in the text.
    """
    outcome = re.findall(r"[a-zA-Z]+", text.lower())
    return outcome

In [202]:
list_of_lyrics = lyric_df['text'].tolist()

In [203]:
dict_of_lyrics = {i: row for i, row in enumerate(lyric_df.to_dict(orient='records'))}
#A dictionary of dictionaries. 
#Key: Row number. Value: Dict of {artist:..., link:...., song:..., text:...}

In [204]:
def tokenize_lyrics(dict_of_lyrics, tokenize_method: Callable[[str], List[str]]):
    tokenized_lyrics = {}
    nrows = len(lyric_df)
    for i in range(nrows):
        lyrics = dict_of_lyrics[i]['text']
        tokenized_lyrics[i] = tokenize_method(lyrics)
    return tokenized_lyrics

In [205]:
tokenized_lyrics = tokenize_lyrics(dict_of_lyrics, tokenize)
#A dict with the row numbers of lyric_df as keys and the lsit of tokenized lyrics as value

In [206]:

def build_word_song_count(tokenize_method: Callable[[str], List[str]],
    tokenized_lyrics: Dict[int, List[str]]):
    """Returns a dictionary with the row numbers of songs each distinct word appears in as VALUEs and distinct words as KEYS

    Parameters
    ----------
    tokenize_method : Callable[[str], List[str]]
        A method to tokenize a string into a list of strings representing words.
    tokenized_lyrics: 
        A dict with the row numbers of lyric_df as keys and the lsit of tokenized lyrics as value
    Returns
    -------
    Dict[str, int]
        A dictionary of words mapped to the number of songs they appear in.
    """
    song_count = {}
    
    for key in tokenized_lyrics:
        unique_tokens = set(token.casefold() for token in tokenized_lyrics[key])

        for token in unique_tokens:
            if token not in song_count:
                song_count[token] = set()
            song_count[token].add(key)

    return song_count

In [207]:
song_count = build_word_song_count(tokenize, tokenized_lyrics)
# A dict with KEYS as unique words and VALUES as a list of all the songs(their row numbers) the word appears in

In [4]:

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
 
nltk.download('stopwords')
print(stopwords.words('english'))

ModuleNotFoundError: No module named 'nltk'

In [209]:
def remove_stop_words(list_of_stop_words,tokenized_lyrics):
    
    """Returns a dictionary with KEYS as row values and VALUES as lyrics without stop words
    Parameters
    ----------
    list_of_stop_words : A list of stop words(E.g and, so, the....)
    tokenized_lyrics: 
        A dict with the row numbers of lyric_df as KEYS and the list of tokenized lyrics as VALUE
    Returns
    -------
    Dict[str, int]
        A dictionary of row numbers mapped to the cleaned tokenized lyrics.
    """
    
    clean_stop_words = set()
    for word in list_of_stop_words:
        clean_stop_words.update(nltk.word_tokenize(word.lower()))  
    
    stop_word_removed_lyrics = {}
    
    for i, lyrics in tokenized_lyrics.items():
        cleaned_lyrics = [word for word in lyrics if word.lower() not in clean_stop_words]
        stop_word_removed_lyrics[i] = cleaned_lyrics
                
    return stop_word_removed_lyrics

In [210]:
cleaned_tokenized_lyrics = remove_stop_words(stopwords.words('english'),tokenized_lyrics)
#A dictionary of row numbers mapped to the cleaned tokenized lyrics.

In [211]:
def remove_stop_words_input(tokenize,list_of_stop_words,input_words):
    
    """Returns a dictionary with KEYS as row values and VALUES as lyrics without stop words
    Parameters
    ----------
    list_of_stop_words : A list of stop words(E.g and, so, the....)
    input_words: A string(The input)
    Returns
    -------
    A LIST of tokenized words with stop words removed
    """
    list_tokens = tokenize(input_words)
    cleaned_words = [word for word in list_tokens if word not in list_of_stop_words]
    
    return cleaned_words

In [212]:
clean_song_count = build_word_song_count(tokenize, cleaned_tokenized_lyrics)
# A dict with KEYS as unique NON STOP words and VALUES as a list of all the songs(their row numbers) the word appears in

In [213]:

#Long runtime
def create_j_sim_mat(input_dict) -> np.ndarray:
    """Create Jaccard similarity matrix for songs.
    Create Jaccard similarity matrix, a np.ndarray of size (`len(lyric_df)`, `len(lyric_df)`),
    computing the character similarity, where the entry (i, j) indicating the Jaccard similarity
    between the songs `i` and `j`.

    Parameters
    ----------
    input_dict: input dictionary of KEYS as song row numbers and VALUES as their lyrics
    Returns
    -------
    np.ndarray
        The Jaccard similarity matrix of (`number of rows`, `number of rows`), with the entry
        (i, j) indicating the Jaccard similarity between the songs `i` and `j`.
    """
    return_matrix = np.zeros((len(lyric_df),len(lyric_df)))

    for song_index in input_dict:
      for other_song_index in input_dict:
        song_words = set(input_dict[song_index])
        other_song_words = set(input_dict[other_song_index])
        union = song_words | other_song_words
        intersection = song_words & other_song_words
        value = (len(intersection)-1)/(len(union)-1) 
        return_matrix[song_index][other_song_index] = value
    
    return return_matrix

In [214]:

def build_inverted_index(songs: dict) -> dict:
    """Builds an inverted index from the song lyrics.

    Arguments
    =========

    songs: dict
        A dictionary where keys are song numbers and values are lists of tokens (words from the lyrics).

    Returns
    =======

    inverted_index: dict
        An inverted index mapping each term to a sorted list of tuples (song_number, term_frequency):
        inverted_index[term] = [(s1, tf1), (s2, tf2), ...]

    Example
    =======

    >> songs = {
    ...    1: ['love', 'me', 'do', 'love', 'love'],
    ...    2: ['hello', 'goodbye', 'hello']
    ... }

    >> idx = build_inverted_index(songs)

    >> idx['love']
    [(1, 3),(2,2)....]

    >> idx['hello']
    [(2, 2)]
    """
    inverted_index = {}

    for song_num, tokens in songs.items():
        token_counts = {}
        
        for token in tokens:
            token_counts[token] = token_counts.get(token, 0) + 1
        
        for token, count in token_counts.items():
            if token not in inverted_index:
                inverted_index[token] = []
            inverted_index[token].append((song_num, count))

    return inverted_index


In [215]:
import math
def compute_idf(inv_idx, n_docs, min_df=0, max_df_ratio=1):
    """Compute term IDF values from the inverted index.
    Words that are too frequent or too infrequent get pruned.

    Hint: Make sure to use log base 2.

    inv_idx: an inverted index as above

    n_docs: int,
        The number of songs.

    min_df: int,
        Minimum number of documents a term must occur in.
        Less frequent words get ignored.
        Documents that appear min_df number of times should be included.

    max_df_ratio: float,
        Maximum ratio of documents a term can occur in.
        More frequent words get ignored.

    Returns
    =======

    idf: dict
        For each term, the dict contains the idf value.

    """

    # TODO-5.1
    return_dict = {}
    
    for word, word_list in inv_idx.items():
        word_count = len(word_list)
        
        if word_count >= min_df and word_count / n_docs <= max_df_ratio:
            idf = math.log2(n_docs/(1+word_count))
            return_dict[word] = idf
                
    return return_dict

In [240]:
inverted_index = build_inverted_index(cleaned_tokenized_lyrics)

In [217]:
tdidf = compute_idf(inverted_index, len(lyric_df), min_df=0, max_df_ratio=1)

In [219]:
def freq_dis(list_of_words):
    """How many times each word appears within a given text. 
    
    Parameters
    ----------
    list_of_words: a list of tokenized words
    """
    fd = nltk.FreqDist(list_of_words)
    return fd

In [220]:
text = nltk.Text(cleaned_tokenized_lyrics[0])
text.concordance("mine", lines=5)

Displaying 2 of 2 matches:
nd girl makes feel fine ever believe mine kind girl without blue ever leaves g
nd girl makes feel fine ever believe mine kind girl without blue ever leaves


In [221]:
#Bigrams: Frequent two-word combinations
#Trigrams: Frequent three-word combinations
#Quadgrams: Frequent four-word combinations
finder = nltk.collocations.TrigramCollocationFinder.from_words(text)
result = finder.ngram_fd.most_common(2)


In [222]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/juanruzhang/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [223]:
sia.polarity_scores("Wow, NLTK is not not not terrible!")
# compound ranges from -1 (most negative) to +1 (most positive).

{'neg': 0.0, 'neu': 0.457, 'pos': 0.543, 'compound': 0.7134}

In [224]:
def classify_word_sentiments(list_of_words):
    """_summary_

    Args:
        list_of_words (_type_): list of words

    Returns:
        a dictionary with KEYS as positive, negative, neutral and VALUES of sets of words that belong to each category
    """
    positive = []
    negative = []
    neutral = []

    for word in list_of_words:
        score = sia.polarity_scores(word)["compound"]
        if score > 0:
            positive.append(word)
        elif score < 0:
            negative.append(word)
        else:
            neutral.append(word)
    
    return {"positive":positive, "negative":negative, "neutral":neutral}


In [225]:
from itertools import chain
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [226]:
def get_synonyms_of_word(word):
    """_summary_

    Args:
        word (_type_): word

    Returns:
        _type_: a set of all the synonyms of the word
    """
    synonyms = []

    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            synonyms.append(i.name().replace("_", " ").lower())

    return set(synonyms)

In [ ]:
for key in cleaned_tokenized_lyrics:
    if "love you" in " ".join(cleaned_tokenized_lyrics[key]):
        #print(key)
        continue

1749
2285
2732
3226
3337
3378
9015
9557
9581
10116
10518
11146
11169
11898
12131
13034
13342
13643
15670
15914
15933
17241
17406
17763
18303
20325
20507
21311
24165
24533
24769
25883
26151
26910
27062
27114
27259
27769
30687
31697
34153
36810
37041
37267
40140
41659
41665
43227
43810
49969
50887
51074
51129
52904
52991
53235
53290
55768
56248
56836


In [228]:
# Stemming words
ps = PorterStemmer()
ps.stem("words")

'word'

ACTUAL IMPLEMENTATION START HERE
Step 1: Get inputs
2: Use genre to search for songs
3: Find exact matching words 

In [337]:
from collections import Counter
# Find exact matching words 
# "genre_input" is the user inputted genre
user_genre_input = "gorge"

#Tokenize and clean(Remove stop words) user input
#clean_genre_input : {0:[Cleaned user input]}
clean_genre_input = remove_stop_words(stopwords.words('english'),{0:tokenize(user_genre_input)})

#A dict with KEYS as unique NON STOP words !!from the user input!! and VALUES as a list of all the songs(their row numbers) the word appears in
possible_songs_dict = {}

for word in clean_genre_input[0]:
    if clean_song_count.get(word) is not None:
        #Clean_song_count: A dict with KEYS as unique NON STOP words and VALUES as a list of all the songs(their row numbers) the word appears in
        possible_songs_dict[word] = clean_song_count[word]
print(possible_songs_dict.keys())
    
#Stem the words that can't be found in song lyrics and add them to stemmed_user_input
stemmed_user_input = []
#Stem the words that can't be found in song lyrics and add them to stemmed_user_input, preserve words that are already found in song lyrics
stemmed_user_input_preserve_original = []
for word in clean_genre_input[0]:
    if possible_songs_dict.get(word) is None:
        stemmed_user_input_word = ps.stem(word)
        stemmed_user_input.append(stemmed_user_input_word)
        stemmed_user_input_preserve_original.append(stemmed_user_input_word)
    else:
        stemmed_user_input_preserve_original.append(word)

#Update the list of possible songs by searching for stemmed words in the songs
for word in stemmed_user_input:
    if clean_song_count.get(word) is not None:
        possible_songs_dict[word] = clean_song_count[word]
print(possible_songs_dict.keys())

#Get the songs that has the most number of relevant words from user input
most_common_songs = []
if possible_songs_dict:
    # Count occurrences of each song index
    song_counts = Counter(song for song_list in possible_songs_dict.values() for song in song_list)
    # Find the maximum count
    max_number = max(song_counts.values(), default=0)
    # Return the song indices that appear the most
    most_common_songs = [song for song, count in song_counts.items() if count == max_number]

#For words in the user input that don't exist in any songs, find the synonyms
# A dictionary with KEYS as user input words that don't exist in any songs, VALUES as a list of their synonyms
word_synonym_dict = {}
for word in stemmed_user_input_preserve_original:
    if possible_songs_dict.get(word) is None:
        set_of_synonyms = get_synonyms_of_word(word)
        word_synonym_dict[word] = list(set_of_synonyms)
print(word_synonym_dict)
##NEED TO FIX: Not sure how to incorporate synonyms


dict_keys(['gorge'])
dict_keys(['gorge'])
{}


In [338]:
print(most_common_songs)

[53789, 28893, 53663]


In [339]:
#Get the total number(including repeats) of inputted words in song
if len(most_common_songs) > 40:
    most_common_songs = most_common_songs[:40]  

sum_of_words_dict = {}

for word in clean_genre_input[0]:
    if word not in inverted_index:
        continue  

    inv_index_list = inverted_index[word]
    for song_idx, num_words in inv_index_list:
        if song_idx not in most_common_songs:
            continue

        sum_of_words_dict[song_idx] = sum_of_words_dict.get(song_idx, 0) + num_words

top_30_songs = sorted(sum_of_words_dict, key=sum_of_words_dict.get, reverse=True)[:30]


In [340]:
print(top_30_songs)

[53789, 28893, 53663]


In [341]:
#Find songs without(or with very little) antonyms
def get_antonyms(word):
    antonyms = []
    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            if i.antonyms():
                antonyms.append(i.name().replace("_", " ").lower())
    return set(antonyms)

In [342]:
# Step 1: Get all antonyms for user input words
word_antonyms = {word: get_antonyms(word) for word in clean_genre_input[0]}

# for the top_30_songs, find the songs with these antonyms
song_antonym_counts = Counter()

for word, antonyms in word_antonyms.items():
    for antonym in antonyms:
        if antonym in clean_song_count:
            for song in clean_song_count[antonym]:
                if song in top_30_songs: 
                    song_antonym_counts[song] += 1 

# Step 3: Add songs with no antonyms with values of 0
for song in top_30_songs:
    if song not in song_antonym_counts:
        song_antonym_counts[song] = 0

# Step 4: Sort songs by least distinct antonyms
filtered_songs = sorted(song_antonym_counts.keys(), key=lambda song: song_antonym_counts[song])

filtered_songs = filtered_songs[:10]



In [327]:
print(filtered_songs)

[10853, 13130, 41152, 44258, 55572]


In [343]:

#Sort the input_song_list based on the polarity score(pos, neg, neutral, compound(Compound being -1 to 1, with -1 being neg and 1 being pos))
def sort_polarity_scores(input_song_list, cleaned_tokenized_lyrics, polarity_type="compound"):
    sia = SentimentIntensityAnalyzer() 
    dict_of_polarity_scores = {}

    for song_row_number in input_song_list:
        if song_row_number in cleaned_tokenized_lyrics:
            dict_of_polarity_scores[song_row_number] = sia.polarity_scores(" ".join(cleaned_tokenized_lyrics[song_row_number]))

    # Sort songs by the specified polarity score in descending order
    sorted_songs = sorted(dict_of_polarity_scores, key=lambda x: dict_of_polarity_scores[x][polarity_type], reverse=True)

    return sorted_songs

In [344]:
#Takes too long to run
result = sort_polarity_scores(filtered_songs, cleaned_tokenized_lyrics, "pos")

In [345]:
print(result)

[28893, 53789, 53663]


In [346]:
print(cleaned_tokenized_lyrics[28893])

['david', 'allan', 'coe', 'dedication', 'told', 'george', 'jones', 'full', 'whiskey', 'sing', 'anymore', 'said', 'skinny', 'sickly', 'looking', 'told', 'em', 'one', 'best', 'friends', 'whiskey', 'never', 'make', 'gorge', 'jones', 'quit', 'singing', 'george', 'wagon', 'done', 'song', 'sings', 'like', 'wishes', 'drink', 'last', 'week', 'spent', 'whole', 'pay', 'check', 'whiskey', 'know', 'friday', 'night', 'oh', 'drink', 'till', 'falls', 'order', 'one', 'round', 'go', 'home', 'bottle', 'hand', 'thing', 'hold', 'bottle', 'hand', 'know', 'never', 'share', 'man', 'glad', 'introduced', 'us', 'best', 'faithful', 'bottle', 'hand', 'told', 'george', 'changing', 'diapers', 'sure', 'become', 'drag', 'clean', 'house', 'cook', 'kind', 'bag', 'said', 'needed', 'someone', 'love', 'understand', 'left', 'bottle', 'hand', 'thing', 'hold', 'bottle', 'hand', 'know', 'never', 'share', 'man', 'glad', 'introduced', 'us', 'best']
